# Dihedral symmetry of the Robinson form

**Adapted from**: Example 5.4 of [GP04]

[GP04] Gatermann, Karin and Parrilo, Pablo A.
*Symmetry groups, semidefinite programs, and sums of squares*.
Journal of Pure and Applied Algebra 192.1-3 (2004): 95-128.

We start by defining the Dihedral group of order 8.
This group is isomorphic to the following permutation group:

In [1]:
using PermutationGroups
d = perm"(1, 2, 3, 4)"
c = perm"(1, 3)"
G = PermGroup([c, d])

Permutation group on 2 generators generated by
 (1,3)
 (1,2,3,4)

We could rely on this isomorphism to define this group.
However, in order to illustrate how to do symmetry reduction with a custom group,
we show in this example what should be implemented to define a new group.

In [2]:
import GroupsCore
struct DihedralElement <: GroupsCore.GroupElement
    n::Int
    reflection::Bool
    id::Int
end
function Base.:(==)(g::DihedralElement, h::DihedralElement)
    return g.n == h.n && g.reflection == h.reflection && g.id == h.id
end
function PermutationGroups.order(el::DihedralElement)
    if el.reflection
        return 2
    else
        if iszero(el.id)
            return 1
        else
            return div(el.n, gcd(el.n, el.id))
        end
    end
end
Base.one(el::DihedralElement) = DihedralElement(el.n, false, 0)
function Base.inv(el::DihedralElement)
    if el.reflection || iszero(el.id)
        return el
    else
        return DihedralElement(el.n, false, el.n - el.id)
    end
end
function Base.:*(a::DihedralElement, b::DihedralElement)
    a.n == b.n || error("Cannot multiply elements from different Dihedral groups")
    id = mod(a.reflection ? a.id - b.id : a.id + b.id, a.n)
    return DihedralElement(a.n, a.reflection != b.reflection, id)
end
function Base.:^(el::DihedralElement, k::Integer)
    if el.reflection
        return iseven(k) ? one(el) : el
    else
        return DihedralElement(el.n, false, mod(el.id * k, el.n))
    end
end

Base.conj(a::DihedralElement, b::DihedralElement) = inv(b) * a * b
Base.:^(a::DihedralElement, b::DihedralElement) = conj(a, b)

struct DihedralGroup <: GroupsCore.Group
    n::Int
end
Base.one(G::DihedralGroup) = DihedralElement(G.n, false, 0)
PermutationGroups.gens(G::DihedralGroup) = [DihedralElement(G.n, false, 1), DihedralElement(G.n, true, 0)]
PermutationGroups.order(::Type{T}, G::DihedralGroup) where {T} = convert(T, 2G.n)
function Base.iterate(G::DihedralGroup, prev::DihedralElement=DihedralElement(G.n, false, -1))
    if prev.id + 1 >= G.n
        if prev.reflection
            return nothing
        else
            next = DihedralElement(G.n, true, 0)
        end
    else
        next = DihedralElement(G.n, prev.reflection, prev.id + 1)
    end
    return next, next
end

The Robinson form is invariant under the following action of the Dihedral group on monomials:
The action of each element of the groups is to map the variables `x, y` to:

| id | rotation | reflection |
|----|----------|------------|
| 0  | x, y     | y, x       |
| 1  | -y, x    | -x, y      |
| 2  | -x, -y   | -y, -x     |
| 3  | y, -x    | x, -y      |

In [3]:
using SumOfSquares
using DynamicPolynomials
@polyvar x y
struct DihedralAction <: Symmetry.OnMonomials end
import SymbolicWedderburn
SymbolicWedderburn.coeff_type(::DihedralAction) = Float64
function SymbolicWedderburn.action(::DihedralAction, el::DihedralElement, mono::AbstractMonomial)
    if iseven(el.reflection + el.id)
        var_x, var_y = x, y
    else
        var_x, var_y = y, x
    end
    sign_x = 1 <= el.id <= 2 ? -1 : 1
    sign_y = 2 <= el.id ? -1 : 1
    return mono([x, y] => [sign_x * var_x, sign_y * var_y])
end

poly = x^6 + y^6 - x^4 * y^2 - y^4 * x^2 - x^4 - y^4 - x^2 - y^2 + 3x^2 * y^2 + 1

x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1

We can verify that `poly` is indeed invariant under the action of each element of the group as follows.

In [4]:
G = DihedralGroup(4)
for g in G
    @show SymbolicWedderburn.action(DihedralAction(), g, poly)
end

SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1


We can exploit this symmetry for reducing the problem using the `SymmetricIdeal` certificate as follows:

In [5]:
import CSDP
function solve(G)
    solver = CSDP.Optimizer
    model = Model(solver)
    @variable(model, t)
    @objective(model, Max, t)
    pattern = Symmetry.Pattern(G, DihedralAction())
    con_ref = @constraint(model, poly - t in SOSCone(), symmetry = pattern)
    optimize!(model)
    @show value(t)


    for g in gram_matrix(con_ref).sub_gram_matrices
        println(g.basis.polynomials)
    end
end
solve(G)

R * basis.monomials = DynamicPolynomials.Polynomial{true, Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}[x³, x²y, xy², y³, x, y]
m = 3
d = 2
R * basis.monomials = DynamicPolynomials.Polynomial{true, Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}[x² + y², ( 1.0*E(1)^0)]
m = 2
d = 1
R * basis.monomials = DynamicPolynomials.Polynomial{true, Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}[xy]
m = 1
d = 1
R * basis.monomials = DynamicPolynomials.Polynomial{true, Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}[x² + -y²]
m = 1
d = 1
Iter: 17 Ap: 9.60e-01 Pobj: -3.5512010e-03 Ad: 9.60e-01 Dobj: -3.5513625e-03 
Success: SDP solved
Primal objective value: -3.5512010e-03 
Dual objective value: -3.5513625e-03 
Relative primal infeasibility: 2.04e-13 
Relative dual infeasibility: 1.65e-09 
Real Relative Gap: -1.60e-07 
XZ Relative Gap: 2.91e-09 
DIMACS error measures: 2.20e-13 0.00e+00 

6×10 Matrix{Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0

2×10 Matrix{Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}:
 0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

1×10 Matrix{Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}:
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0

1×10 Matrix{Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}:
 0.0  0.0  0.0  0.0  1.0  0.0  -1.0  0.0  0.0  0.0

We notice that we indeed find `-3825/4096` and that symmetry was exploited.
In case the conjugacy classes are known, we can implement
`SymbolicWedderburn.conjugacy_classes_orbit` instead of `order` and `iterate`.
To show that these do not need to be implemented, we create a new dihedral group type
that do not implement these methods but that instead implement
`SymbolicWedderburn.conjugacy_classes_orbit`:

In [6]:
struct DihedralGroup2 <: GroupsCore.Group
    n::Int
end
PermutationGroups.gens(G::DihedralGroup2) = [DihedralElement(G.n, false, 1), DihedralElement(G.n, true, 0)]
_orbit(cc::Vector{<:GroupsCore.GroupElement}) = PermutationGroups.Orbit(cc, Dict(a => nothing for a in cc))
_orbit(el::GroupsCore.GroupElement) = _orbit([el])
function SymbolicWedderburn.conjugacy_classes_orbit(d::DihedralGroup2)
    orbits = [_orbit(DihedralElement(d.n, false, 0))]
    for i in 1:div(d.n - 1, 2)
        push!(orbits, _orbit([
            DihedralElement(d.n, false, i),
            DihedralElement(d.n, false, d.n - i),
        ]))
    end
    if iseven(d.n)
        push!(orbits, _orbit(DihedralElement(d.n, false, div(d.n, 2))))
        push!(orbits, _orbit([
            DihedralElement(d.n, true, i) for i in 0:2:(d.n - 2)
        ]))
        push!(orbits, _orbit([
            DihedralElement(d.n, true, i) for i in 1:2:(d.n - 1)
        ]))
    else
        push!(orbits, _orbit([
            DihedralElement(d.n, true, i) for i in 0:(d.n - 1)
        ]))
    end
end

As we have not implemented the iterator over all the elements, we can iterate over all
conjugacy classes instead to verify that the polynomial is invariant under the group action.

In [7]:
G = DihedralGroup2(4)
for cc in SymbolicWedderburn.conjugacy_classes_orbit(G)
    for g in cc
        @show SymbolicWedderburn.action(DihedralAction(), g, poly)
    end
end

solve(G)

SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
SymbolicWedderburn.action(DihedralAction(), g, poly) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
R * basis.monomials = DynamicPolynomials.Polynomial{true, Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int

6×10 Matrix{Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0

2×10 Matrix{Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}:
 0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

1×10 Matrix{Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}:
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0

1×10 Matrix{Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}:
 0.0  0.0  0.0  0.0  1.0  0.0  -1.0  0.0  0.0  0.0

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*